In [ ]:
! pip install numpy
! pip install openai==1.2.3
! pip install pymongo
! pip install python-dotenv
! pip install prettytable

## Set up

In [162]:
from prettytable import PrettyTable
import openai
import pymongo

from dotenv import dotenv_values
from openai import AzureOpenAI

env_name = "myconfig.env" 
config = dotenv_values(env_name)

# Values for the two prediction functions
manager_query = "Snowboards used by Olympic champions"
manager_id = int(189)
element_id = int(92) # Shaun White snowboard
num_results = 10


# Connection string
cosmos_conn = config['cosmos_connection_string']
cosmos_client = pymongo.MongoClient(cosmos_conn)

# Database name
database = cosmos_client[config['cosmos_database']]

# Collection names
actual_nodes = database[config['cosmos_actual_nodes']]
predicted_nodes = database[config['cosmos_predicted_nodes']]
element_catalog = database[config['cosmos_element_catalog']]

# OpenAI
openai.api_type = config['openai_type']
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_endpoint']
openai.api_version = config['openai_api_version']

client = AzureOpenAI(
    api_key=openai.api_key,
    api_version=openai.api_version,
    azure_endpoint = openai.api_base
)

In [139]:
def generate_embeddings(text):
    
    response = client.embeddings.create(
        input=text, model="embeddings") # need to read from config
    
    embeddings = response.data[0].embedding
        
    return embeddings

In [163]:
def print_vector_search_results(results):
    
    print("---------Vector Search Results: --------")

    # Define the table
    table = PrettyTable()
    table.field_names = ["element Id", "Name", "Cost", "Similarity Score", "node"]

    # Add rows to the table
    for element in results:
        table.add_row([
            element['document']['Id'],
            element['document']['Name'],
            element['document']['Cost'],
            element['similarityScore'],
            element['document']['node']
        ])

    # Print the table
    print(table)

In [166]:
def print_predictions_from_element_page(results):
    print("\n--------Current Page Results: ---------")

    # Define the table
    table = PrettyTable()
    table.field_names = ["element Id", "Name", "Cost", "node"]

    # Add rows to the table
    for element in results:
        table.add_row([
            element['Id'],
            element['Name'],
            element['Cost'],
            element['node']
        ])

    # Print the table
    print(table)

## element Recommendation Functions

In [176]:
def predictions_from_current_element_page(manager_id, current_element_id, num_results=4):
    """
    This function displays predicted elements for this manager excluding the current element.
    """
    
    # Get the predicted elements for the manager, limit results
    manager_predicted_elements = predicted_nodes.find_one( 
        { "ManagerId": manager_id },
        {"Predictions": {"$slice": num_results}})

    # Remove the current element from the list
    manager_predicted_elements = [prediction for prediction in manager_predicted_elements['Predictions']
        if prediction['elementId'] != current_element_id]
    
    predicted_elements = []

    # Look up recommended elements maintaining order of predicted nodes
    for item in manager_predicted_elements:
        element = element_catalog.find_one({"Id": item['elementId']})
        if element:
            predicted_elements.append(element)
            predicted_elements[-1]['node'] = item['node']

    predicted_elements = list(predicted_elements)

    return predicted_elements

In [175]:
# Test the Function above on predictions excluding the current element on page
manager_id = int(189)
element_id = int(92) # Shaun White snowboard
num_results = 10

# Predictions excluding the current element on page
on_page_predictions = predictions_from_current_element_page(manager_id, element_id, num_results)
print_predictions_from_element_page(on_page_predictions)



--------Current Page Results: ---------
+------------+--------------------------------+--------+-------------------+
| element Id |              Name              | Cost  |       node      |
+------------+--------------------------------+--------+-------------------+
|     42     | Gravity 5000 All-Mountain Skis | 699.0  |  7.15698766708374 |
|     72     | GravityZone All-Mountain Skis  | 699.0  | 7.150111198425293 |
|     22     |     Venture 2022 Snowboard     | 499.0  | 6.249397277832031 |
|     62     |     Shadow Black Snowboard     | 379.0  | 5.958791732788086 |
|     27     |  EcoLodge 45L Travel Backpack  | 129.0  | 5.905089378356934 |
|     88     |   Alpine AlpinePack Backpack   | 129.0  | 5.895949363708496 |
|     32     |    Cosmic Purple Snowboard     | 419.99 | 5.894195079803467 |
|     53     |     Raven Swift Snowboard      | 349.0  | 5.818643569946289 |
|     73     |    Omni-Snow Dual Snowboard    | 289.99 | 5.817547798156738 |
+------------+------------------------

In [192]:
def predictions_from_vector_search(manager_id, manager_query, num_results=10):
    """ 
    This function takes a manager prompt search for elements and returns elements that are predicted for the manager. 
    """
    
    # Generate the embedding for the manager query
    query_embedding = generate_embeddings(manager_query)

    # Get the predicted elements for the manager
    predicted_elements = predicted_nodes.find_one( { "ManagerId": manager_id } )

    # Convert to a dictionary
    predicted_elements = {prediction['elementId']: prediction for prediction in predicted_elements['Predictions']}

    # Filter criteria to include element ids from the predicted elements
    filter_criteria = { 
        "Id": {"$in": list(predicted_elements.keys())}
    }

    results = element_catalog.aggregate([
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "Embedding",
                    "k": num_results,
                    "filter": filter_criteria
                },
                "returnStoredSource": True
            }},
        {
            '$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' }
        }
    ])

    filtered_vector_search = list(results)

    # Add the node field to the documents in filtered_vector_search
    for document in filtered_vector_search:
        element_id = document['document']['Id']
        if element_id in predicted_elements:
            document['document']['node'] = predicted_elements[element_id]['node']

    # Remove the top vector search result. Add back after sorting by node
    top_vector_result = filtered_vector_search.pop(0)
    
    # Sort the remaining results by node
    sorted_vector_search = sorted(
        filtered_vector_search,
        key=lambda document: (-document['document'].get('node', 0)),
        reverse=False
    )

    # Insert the top result at the beginning of the list
    sorted_vector_search.insert(0, top_vector_result)

    return sorted_vector_search

In [193]:
# Test the function above for filtering vector search results with predicted elements

manager_query = "Node closest to element"
manager_id = int(189)
num_results = 10

# Vector Search with Predictions
vector_search_with_predictions = predictions_from_vector_search(manager_id, manager_query, num_results)
print_vector_search_results(vector_search_with_predictions)


---------Vector Search Results: --------
+------------+------------------------------+--------+--------------------+--------------------+
| element Id |             Name             | Cost  |  Similarity Score  |       node       |
+------------+------------------------------+--------+--------------------+--------------------+
|     92     |             Element1         | 449.99 | 0.8480039834976196 | 6.024456977844238  |
|     22     |             Element2         | 499.0  | 0.8239901485466837 | 5.9668755531311035 |
|     62     |             Element3         | 379.0  | 0.8160758261703205 | 5.747269630432129  |
|     32     |             Element4         | 419.99 | 0.8276490959702903 | 5.633471965789795  |
|     53     |             Element5         | 349.0  | 0.8259191409301261 | 5.577742099761963  |
|     73     |             Element6         | 289.99 | 0.8393915199870792 | 5.572394847869873  |
|     60     |             Element7         | 249.0  | 0.8236519484105107 | 5.37770986557